In [5]:
import os
from openai import OpenAI

In [6]:
from dotenv import load_dotenv
load_dotenv()

token = os.getenv("GITHUB_TOKEN")
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4.1-mini"

In [7]:
client = OpenAI(
    base_url=endpoint,
    api_key=token,
)

In [11]:
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
    ],
    temperature=1.0,
    top_p=1.0
)


In [16]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role":"user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model=model,
            temperature=0,
            messages=self.messages
        )
        return completion.choices[0].message.content
            

In [17]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [18]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [19]:
abot = Agent(prompt)

In [20]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I need to find the average weight of a Toy Poodle using the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


In [22]:
result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

In [23]:
next_prompt = "Observation: {}".format(result)

In [25]:
abot(next_prompt)

'Answer: A Toy Poodle weighs on average 7 lbs.'

In [26]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [27]:
abot = Agent(prompt)

In [28]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add them together to get the combined weight.\nAction: average_dog_weight: Border Collie\nPAUSE'

In [29]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [30]:
abot(next_prompt)

'Thought: I have the average weight of a Border Collie as 37 lbs. Now I need to find the average weight of a Scottish Terrier.\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [31]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [32]:
abot(next_prompt)

'Thought: The average weight of a Border Collie is 37 lbs and the average weight of a Scottish Terrier is 20 lbs. I will add these two weights to get the combined weight.\nAction: calculate: 37 + 20\nPAUSE'

In [33]:
next_prompt = "Observation: {}".format(eval("37+20"))
print(next_prompt)

Observation: 57


In [34]:
abot(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.'

In [35]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [38]:
def query(question, max_turns=5):
    i = 0  # Counter to track how many turns (iterations) we’ve taken
    bot = Agent(prompt)  # Initialize the chatbot/agent with a prompt
    next_prompt = question  # Start the conversation with the given question

    while i < max_turns:  # Limit the loop to avoid infinite conversations
        i += 1  # Increment turn counter

        result = bot(next_prompt)  # Send the prompt to the bot and get its response
        print(result)  # Print the bot’s response to the console

        # Try to extract any "Action(param)" lines from the response
        actions = [
            action_re.match(a)  # Use regex to match lines that represent actions
            for a in result.split('\n')  # Split the response line-by-line
            if action_re.match(a)  # Only keep lines that match the action format
        ]
        actions = [a for a in actions if a]  # Remove any None values (unsuccessful matches)

        if actions:
            # An action was found in the response
            action, action_input = actions[0].groups()  # Extract action name and input from the first match

            # Check if this action is registered (known)
            if action not in known_actions:
                # If not, throw an error with a helpful message
                raise Exception("Unknown action: {}: {}".format(action, action_input))

            # Log what action we're about to run
            print(" -- running {} {}".format(action, action_input))

            # Run the corresponding action function with the input and get its output
            observation = known_actions[action](action_input)

            # Log the result (observation) from the action
            print("Observation:", observation)

            # Feed the observation back into the bot as the next prompt
            next_prompt = "Observation: {}".format(observation)
        else:
            # No action to take, end the conversation
            return


In [39]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add them together to get the combined weight.
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: I have the average weight of a Border Collie as 37 lbs. Now I need to find the average weight of a Scottish Terrier to calculate the combined weight.
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: I have the average weights of both dogs: Border Collie is 37 lbs and Scottish Terrier is 20 lbs. I will add these two weights to get the combined weight.
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.
